In [2]:
import csv
train_texts = []
train_labels = []
with open('data/banking_data/train.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    headers = next(reader, None)
    for row in reader:
        train_texts.append(row[0])
        train_labels.append(row[1])

        
test_texts = []
test_labels = []
with open('data/banking_data/test.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    headers = next(reader, None)
    for row in reader:
        test_texts.append(row[0])
        test_labels.append(row[1])

In [3]:
BATCH_SIZE = 32
MAX_LEN = 250

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch


if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
    
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

There are 2 GPU(s) available.
Device name: GeForce RTX 2080 Ti


In [5]:
import nltk
# Uncomment to download "stopwords"
# nltk.download("stopwords")
from nltk.corpus import stopwords
import re

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))
        
    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks


In [6]:
train_texts = [text_preprocessing(text) for text in train_texts]
test_texts = [text_preprocessing(text) for text in test_texts]
train_inputs, train_masks = preprocessing_for_bert(train_texts)
test_inputs, test_masks = preprocessing_for_bert(test_texts)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/tuandh/PycharmProjects/research-chatbot/venv/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [7]:
list_categories = [
  "card_arrival",
  "card_linking",
  "exchange_rate",
  "card_payment_wrong_exchange_rate",
  "extra_charge_on_statement",
  "pending_cash_withdrawal",
  "fiat_currency_support",
  "card_delivery_estimate",
  "automatic_top_up",
  "card_not_working",
  "exchange_via_app",
  "lost_or_stolen_card",
  "age_limit",
  "pin_blocked",
  "contactless_not_working",
  "top_up_by_bank_transfer_charge",
  "pending_top_up",
  "cancel_transfer",
  "top_up_limits",
  "wrong_amount_of_cash_received",
  "card_payment_fee_charged",
  "transfer_not_received_by_recipient",
  "supported_cards_and_currencies",
  "getting_virtual_card",
  "card_acceptance",
  "top_up_reverted",
  "balance_not_updated_after_cheque_or_cash_deposit",
  "card_payment_not_recognised",
  "edit_personal_details",
  "why_verify_identity",
  "unable_to_verify_identity",
  "get_physical_card",
  "visa_or_mastercard",
  "topping_up_by_card",
  "disposable_card_limits",
  "compromised_card",
  "atm_support",
  "direct_debit_payment_not_recognised",
  "passcode_forgotten",
  "declined_cash_withdrawal",
  "pending_card_payment",
  "lost_or_stolen_phone",
  "request_refund",
  "declined_transfer",
  "Refund_not_showing_up",
  "declined_card_payment",
  "pending_transfer",
  "terminate_account",
  "card_swallowed",
  "transaction_charged_twice",
  "verify_source_of_funds",
  "transfer_timing",
  "reverted_card_payment?",
  "change_pin",
  "beneficiary_not_allowed",
  "transfer_fee_charged",
  "receiving_money",
  "failed_transfer",
  "transfer_into_account",
  "verify_top_up",
  "getting_spare_card",
  "top_up_by_cash_or_cheque",
  "order_physical_card",
  "virtual_card_not_working",
  "wrong_exchange_rate_for_cash_withdrawal",
  "get_disposable_virtual_card",
  "top_up_failed",
  "balance_not_updated_after_bank_transfer",
  "cash_withdrawal_not_recognised",
  "exchange_charge",
  "top_up_by_card_charge",
  "activate_my_card",
  "cash_withdrawal_charge",
  "card_about_to_expire",
  "apple_pay_or_google_pay",
  "verify_my_identity",
  "country_support"
]
def label2id(label):
    return list_categories.index(label)
def id2label(id_):
    return list_categories[id_]

In [8]:
train_labels = [label2id(label) for label in train_labels]
test_labels = [label2id(label) for label in test_labels]
# Convert other data types to torch.Tensor
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

In [9]:
import torch

print(train_inputs.shape)
print(test_inputs.shape)
print(train_labels.shape)
print(test_labels.shape)

torch.Size([10003, 250])
torch.Size([3080, 250])
torch.Size([10003])
torch.Size([3080])


In [19]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE, shuffle=False)

# Create the DataLoader for our validation set
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE, shuffle=False)

In [11]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', 
                                                        num_labels = len(list_categories), # The number of output labels--2 for binary classification.
                                                                        # You can increase this for multi-class tasks.   
                                                        output_attentions = False, # Whether the model returns attentions weights.
                                                        output_hidden_states = False) # Whether the model returns all hidden-states.)

model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [14]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
    
for param in model.bert.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True
    print(param)

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [17]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
from transformers import AdamW
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
from transformers import get_linear_schedule_with_warmup
epochs = 4

total_steps = len(train_dataloader) * epochs
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [20]:
import random
import numpy as np
from tqdm import tqdm
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


epochs = 15

# Store the average loss after each epoch so we can plot them.
loss_values = []
# For each epoch...
for epoch_i in range(0, epochs):
    
        # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    model.train()
    # For each batch of training data...
    for step, batch in enumerate(tqdm(train_dataloader)):
        
        total_loss = 0
        
        model.zero_grad()
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        outputs = model(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask, 
            labels=b_labels)
        
        loss = outputs[0]
        
        total_loss += loss.item()
        
        optimizer.zero_grad()
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
#         for param in model.classifier.parameters():
#             param.requires_grad = True
#             print(param)
    avg_train_loss = total_loss / len(train_dataloader)            
    
    loss_values.append(avg_train_loss)
    
    print('Loss values: ')
    print(np.mean(np.array(loss_values)))
    
    
    print("")
    print("Running Validation...")
    
    model.eval()
    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Evaluate data for one epoch
    for batch in test_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
                # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        # Track the number of batches
        nb_eval_steps += 1
    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))

  0%|          | 0/313 [00:00<?, ?it/s]


======== Epoch 1 / 15 ========
Training...


100%|██████████| 313/313 [00:56<00:00,  5.58it/s]


Loss values: 
0.013667519481037371

Running Validation...


  0%|          | 1/313 [00:00<00:57,  5.47it/s]

...

Loss values: 
0.01364058024912651

Running Validation...


  0%|          | 1/313 [00:00<00:57,  5.41it/s]

  Accuracy: 0.03

======== Epoch 8 / 15 ========
Training...


 10%|▉         | 31/313 [00:05<00:53,  5.24it/s]


KeyboardInterrupt: 